In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install transformers
!pip install simpletransformers

In [3]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [4]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.8.1+cu101', '4.6.1', '0.10.3')

In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [6]:
data_file = pd.read_csv("data/trofix.csv")

In [7]:
data_file = data_file.rename(columns={'sentence': 'input_text'}) #abstract - sentence
data_file['target_text'] = data_file[['arg1', 'arg2', 'verb']].agg(' '.join, axis=1) #title - metaphorical words
#data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [8]:
papers = data_file[["input_text", "target_text"]]
papers.head()

,input_text,target_text
0,Triple mileage has struck another blow to the ...,mileage struck blow
1,U.S. officials said evidence suggests that a J...,terrorist attack target
2,"Some police forces , for example , have steppe...",forces stepped use
3,"Every day his troops gather under the green , ...",day pour stream
4,He says manufacturers are increasingly rolling...,manufacturers rolling products


In [9]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

eval_df = papers.sample(frac=0.1, random_state=42)
train_df = papers.drop(eval_df.index)

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 6,
    "num_train_epochs": 3,
}

# Create a Bart-base model
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

CPU times: user 5.08 s, sys: 1.03 s, total: 6.12 s
Wall time: 8.7 s


In [10]:
%%time

# Train the model
model.train_model(train_df)

# Evaluate the model
result = model.eval_model(eval_df)
print(result)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/1300 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/217 [00:00<?, ?it/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/144 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 1.6503019879261653}


{'eval_loss': 1.6503019879261653}
CPU times: user 2min 53s, sys: 3.23 s, total: 2min 57s
Wall time: 2min 47s


In [11]:
for _ in range(65):

    random_idx = random.randint(0, len(eval_df)-1)

    abstract = eval_df.iloc[random_idx]['input_text']
    true_title = eval_df.iloc[random_idx]['target_text']

    # Predict with trained BART model
    predicted_title = model.predict([abstract])[0]

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title: {predicted_title}\n')
    print(f'Abstract: {abstract}\n\n\n')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: kennedy kill newspapers

Predicted Title: k kill newspapers

Abstract: Senator Kennedy , and the handful of legislators who knew about the secret provision , intended to kill two unfriendly newspapers .





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: companies examining possibilities

Predicted Title: compan examining possibilities

Abstract: It said that the companies are examining the possibilities provided within the framework of securities law but that `` all these considerations have n't been in any way concluded . ''





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

True Title: it targeted center

Predicted Title: it targeted center

Abstract: Before midnight , it had targeted the National Aeronautics and Space Administration Ames Research Center in California 's Silicon Valley , as well as the University of Pittsburgh and the Los Alamos National Laboratory in New Mexico .





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: italy dragging feet

Predicted Title: iItaly dragging feet

Abstract: An Aer Lingus spokesman claimed Italy is dragging its feet because Alitalia plans to start flights between Milan and Manchester this month .





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: economy absorb them

Predicted Title: econom absorb them

Abstract: But Korea 's booming economy can absorb them , economists say .



